# Problem 1 (60 points)

In this assignment, you will use a transformer encoder to implement static embeddings (like
word2vec). The method we will use comes from this paper: https://aclanthology.org/2020.acl-
main.431.pdf
Please implement the algorithm below:
1. Choose the transformer encoder you will work with. For example, these are good
encoder models:  
 a. https://huggingface.co/FacebookAI/roberta-base  
 b. https://huggingface.co/microsoft/deberta-v3-base  
 c. https://huggingface.co/Tejas3/distillbert_base_uncased_80_equal  
2. Read the texts provided with this assignment in the dataset uploaded in D2L under
Content / Assignments / assignment4-dataset.txt.gz. This dataset contains one sentence
per line. Tokenize each individual sentence using the tokenizer corresponding to the
transformer chosen in the previous step. Note that the resulting tokens are sub-word
tokens that may not correspond to a full word.
3. Generate the contextualized embeddings for all the tokens in the dataset and compute
the average embedding for each token in the vocabulary by averaging all its
contextualized embeddings.


In [ ]:
%pip install transformers torch tqdm

In [4]:
import torch
import gzip
import numpy as np
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
from collections import defaultdict

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

c:\Users\26653\.conda\envs\torch-nightly\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [ ]:
# Step 1: Choose the transformer encoder
MODEL_NAME = "roberta-base" 

print(f"Loading model: {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval() # Set model to evaluation mode (disables dropout)

# Get hidden size 
hidden_dim = model.config.hidden_size
vocab_size = tokenizer.vocab_size
print(f"Model loaded. Hidden dimension: {hidden_dim}, Vocab size: {vocab_size}")

Loading model: roberta-base...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded. Hidden dimension: 768, Vocab size: 50265


In [ ]:
import torch
import gzip
import os
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# --- 1. Define a Dataset Class for Fast Loading ---
class TextDataset(Dataset):
    def __init__(self, file_path):
        self.lines = []
        open_func = gzip.open if file_path.endswith(".gz") else open
        print(f"Loading {file_path} into memory...")
        
        # Read all lines into memory
        with open_func(file_path, "rt", encoding="utf-8") as f:
            self.lines = [line.strip() for line in f if line.strip()]
        
        # OPTIMIZATION: Sort by length
        # This groups sentences of similar length to minimize padding overhead
        print("Sorting dataset by length for efficiency...")
        self.lines.sort(key=len)

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        return self.lines[idx]

# --- 2. The Optimized Function ---
def compute_static_embeddings_fast(file_path, model, tokenizer, device, batch_size=256):
    dataset = TextDataset(file_path)
    
    # num_workers=0 to avoid multiprocessing overhead
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    token_embedding_sums = torch.zeros((tokenizer.vocab_size, model.config.hidden_size), device=device)
    token_counts = torch.zeros(tokenizer.vocab_size, device=device)

    print(f"Starting High-Performance Inference on {device}...")
    print(f"Batch Size: {batch_size} | Mixed Precision: ON")
    
    model.eval()
    
    for batch_lines in tqdm(loader, unit="batch"):
        inputs = tokenizer(
            batch_lines, 
            return_tensors="pt", 
            padding=True, 
            truncation=True, 
            max_length=512
        ).to(device)

        with torch.no_grad():
            with torch.amp.autocast('cuda'):
                outputs = model(**inputs)
        
        last_hidden_state = outputs.last_hidden_state
        input_ids = inputs.input_ids
        
        # Vectorized Accumulation
        flat_ids = input_ids.view(-1)
        flat_vectors = last_hidden_state.view(-1, model.config.hidden_size)
        
        mask = flat_ids != tokenizer.pad_token_id
        clean_ids = flat_ids[mask]
        clean_vectors = flat_vectors[mask]
        
        token_embedding_sums.index_add_(0, clean_ids, clean_vectors.to(torch.float32))
        
        ones = torch.ones_like(clean_ids, dtype=torch.float)
        token_counts.index_add_(0, clean_ids, ones)

    return token_embedding_sums, token_counts

# --- 3. EXECUTION LOGIC (Smart Caching) ---
DATASET_PATH = "assignment4-dataset.txt" 
OUTPUT_FILE = "roberta_static_embeddings.pt"
BATCH_SIZE = 256 

# Check if file exists first
if os.path.exists(OUTPUT_FILE):
    print(f"✅ Found saved embeddings file: '{OUTPUT_FILE}'")
    print("Loading directly from disk (Skipping computation)...")
    static_embeddings = torch.load(OUTPUT_FILE, map_location=device)

else:
    print(f"❌ File '{OUTPUT_FILE}' not found. Starting computation...")
    
    # Run the fast function
    sums, counts = compute_static_embeddings_fast(DATASET_PATH, model, tokenizer, device, batch_size=BATCH_SIZE)
    
    # Compute Average
    safe_counts = counts.clamp(min=1).unsqueeze(1)
    static_embeddings = sums / safe_counts
    
    # Zero out unseen tokens
    mask = (counts > 0).unsqueeze(1)
    static_embeddings = static_embeddings * mask
    
    # Save to file
    print(f"Saving computed embeddings to '{OUTPUT_FILE}'...")
    torch.save(static_embeddings, OUTPUT_FILE)

# Final Sanity Check
print("-" * 30)
print(f"Final Embeddings Shape: {static_embeddings.shape}")
print("Ready for next steps.")

✅ Found saved embeddings file: 'roberta_static_embeddings.pt'
Loading directly from disk (Skipping computation)...
------------------------------
Final Embeddings Shape: torch.Size([50265, 768])
Ready for next steps.


In [8]:
# Avoid division by zero for tokens that never appeared in the dataset
# We create a clamp to ensure we don't divide by 0 (replace 0 with 1 temporarily)
safe_counts = counts.clamp(min=1).unsqueeze(1)

# Compute averages
static_embeddings = sums / safe_counts

# Zero out embeddings for tokens that were never seen (optional cleanup)
# This ensures unseen tokens are exactly 0 vector rather than a calculation artifact
mask = (counts > 0).unsqueeze(1)
static_embeddings = static_embeddings * mask

print(f"Computed static embeddings matrix shape: {static_embeddings.shape}")

Computed static embeddings matrix shape: torch.Size([50265, 768])


In [ ]:
# Helper function to get embedding for a word
def get_static_embedding(word):
    # Note: Tokenizers might split words or add prefixes (like Ġ in RoBERTa)
    # This is a simplified check for exact token matches
    ids = tokenizer.encode(word, add_special_tokens=False)
    if len(ids) == 1:
        token_id = ids[0]
        count = counts[token_id].item()
        vector = static_embeddings[token_id]
        return token_id, count, vector
    else:
        print(f"Word '{word}' splits into multiple tokens: {ids}. Checking first token.")
        token_id = ids[0]
        count = counts[token_id].item()
        vector = static_embeddings[token_id]
        return token_id, count, vector

t_id, t_count, t_vec = get_static_embedding("film")

print(f"Token ID for 'film': {t_id}")
print(f"Times seen in dataset: {int(t_count)}")
print(f"Embedding vector (first 10 dims): {t_vec[:10].cpu().numpy()}")

# Check if the vector is not all zeros
if torch.count_nonzero(t_vec) > 0:
    print("Success! We have a non-zero static embedding.")
else:
    print("Warning: This token was not found in the dataset.")

Token ID for 'film': 21928
Times seen in dataset: 788
Embedding vector (first 10 dims): [ 0.05221577  0.03705933 -0.01913649 -0.15739763 -0.14939709 -0.0253762
  0.04949529  0.12681545  0.035868    0.05667021]
Success! We have a non-zero static embedding.


In [ ]:
import torch.nn.functional as F

def find_nearest_neighbors(query_word, static_embeddings, tokenizer, top_k=5):
    # 1. Get the ID for the query word
    ids = tokenizer.encode(query_word, add_special_tokens=False)
    if not ids: return
    query_id = ids[0]
    
    # 2. Get the vector for the query word
    query_vec = static_embeddings[query_id].unsqueeze(0) # Shape: [1, 768]
    
    # 3. Calculate Cosine Similarity against ALL other words in vocab 
    sims = F.cosine_similarity(query_vec, static_embeddings)
    
    # 4. Get the top K matches
    top_k_values, top_k_indices = torch.topk(sims, top_k)
    
    print(f"Nearest neighbors for '{query_word}':")
    for value, idx in zip(top_k_values, top_k_indices):
        word = tokenizer.decode([idx]).strip()
        print(f" - {word} (Similarity: {value.item():.4f})")

# Run the test
find_nearest_neighbors("film", static_embeddings, tokenizer)

Nearest neighbors for 'film':
 - film (Similarity: 1.0000)
 - film (Similarity: 0.9594)
 - movie (Similarity: 0.9590)
 - Film (Similarity: 0.9582)
 - artist (Similarity: 0.9509)


# Problem 2 (50 points)

Implement the most_similar() function from the chapter 9 code, and use it to run the six
examples in the notebook. Include the output of these calls in your notebook.
IMPORTANT NOTE: the most_similar() function operates over actual words, whereas the
embeddings you computed in problem 1 operate over transformer tokens. That is, each English
word may consist of one or more tokens. To aggregate token embeddings into word
embeddings, implement the following algorithm:  
  1. Take the glove_vocabulary.txt (available in D2L under Content / Assignments) file and
tokenize all the words in this file using the same tokenizer you used in the previous
problem.  
  2. Compute a word embedding for all words in this file by averaging the corresponding
token embeddings.
Problem

In [3]:
%pip install gensim

   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ------ --------------------------------- 3.9/24.4 MB 18.1 MB/s eta 0:00:02
   ------------ --------------------------- 7.3/24.4 MB 17.4 MB/s eta 0:00:01
   ----------------- ---------------------- 10.5/24.4 MB 17.2 MB/s eta 0:00:01
   ---------------------- ----------------- 13.6/24.4 MB 17.1 MB/s eta 0:00:01
   --------------------------- ------------ 16.8/24.4 MB 16.8 MB/s eta 0:00:01
   -------------------------------- ------- 19.9/24.4 MB 16.6 MB/s eta 0:00:01
   ---------------------------------- ----- 21.0/24.4 MB 16.6 MB/s eta 0:00:01
   ---------------------------------------  24.1/24.4 MB 14.8 MB/s eta 0:00:01
   ---------------------------------------- 24.4/24.4 MB 14.3 MB/s  0:00:01

   ------------- -------------------------- 1/3 [smart_open]
   -------------------------- ------------- 2/3 [gensim]
   -------------------------- ------------- 2/3 [gensim]
   -------------------------- ------------- 2/3 

In [6]:
import numpy as np
from gensim.models import KeyedVectors

# ---------------------------------------------------------
# 1. Load the GloVe Model
# ---------------------------------------------------------
# We need to load the model again because variables don't transfer between notebooks.
print("Loading GloVe model... this may take a minute.")
fname = "glove.6B.300d.txt"

# Load the vectors (no header for GloVe files)
glove = KeyedVectors.load_word2vec_format(fname, binary=False, no_header=True)
print("Model loaded successfully!")

# ---------------------------------------------------------
# 2. Setup Variables
# ---------------------------------------------------------
# Get the normalized vectors for cosine similarity
vectors = glove.get_normed_vectors()
index_to_key = glove.index_to_key
key_to_index = glove.key_to_index

# ---------------------------------------------------------
# 3. Define the most_similar_words function (from Chapter 9)
# ---------------------------------------------------------
def most_similar_words(word, vectors, index_to_key, key_to_index, topn=10):
    # retrieve word_id corresponding to given word
    if word not in key_to_index:
        return [] # Return empty if word not found
        
    word_id = key_to_index[word]
    
    # retrieve embedding for given word
    emb = vectors[word_id]
    
    # calculate similarities to all words in our vocabulary
    similarities = vectors @ emb
    
    # get word_ids in ascending order with respect to similarity score
    ids_ascending = similarities.argsort()
    
    # reverse word_ids to get descending order
    ids_descending = ids_ascending[::-1]
    
    # remove the word itself from the results
    mask = ids_descending != word_id
    ids_descending = ids_descending[mask]
    
    # get topn word_ids
    top_ids = ids_descending[:topn]
    
    # retrieve topn words with their corresponding similarity score
    top_words = [(index_to_key[i], similarities[i]) for i in top_ids]
    
    return top_words

# ---------------------------------------------------------
# 4. Execute the "Six Examples" from the Notebook
# ---------------------------------------------------------
# These are the 6 examples found in the original Chapter 9 notebook
examples = ["cactus", "cake", "angry", "quickly", "between", "the"]

print("-" * 40)
for example in examples:
    print(f"Top 10 words most similar to '{example}':")
    results = most_similar_words(example, vectors, index_to_key, key_to_index)
    
    if not results:
        print(f"  Word '{example}' not found in vocabulary.")
    else:
        for word, score in results:
            print(f"  {word}: {score:.4f}")
    print("-" * 40)

Loading GloVe model... this may take a minute.
Model loaded successfully!
----------------------------------------
Top 10 words most similar to 'cactus':
  cacti: 0.6635
  saguaro: 0.6196
  pear: 0.5233
  cactuses: 0.5178
  prickly: 0.5156
  mesquite: 0.4845
  opuntia: 0.4540
  shrubs: 0.4536
  peyote: 0.4534
  succulents: 0.4513
----------------------------------------
Top 10 words most similar to 'cake':
  cakes: 0.7506
  chocolate: 0.6966
  dessert: 0.6440
  pie: 0.6087
  cookies: 0.6082
  frosting: 0.6017
  bread: 0.5955
  cookie: 0.5934
  recipe: 0.5827
  baked: 0.5820
----------------------------------------
Top 10 words most similar to 'angry':
  enraged: 0.7088
  furious: 0.7078
  irate: 0.6939
  outraged: 0.6705
  frustrated: 0.6516
  angered: 0.6353
  provoked: 0.5827
  annoyed: 0.5819
  incensed: 0.5752
  indignant: 0.5704
----------------------------------------
Top 10 words most similar to 'quickly':
  soon: 0.7662
  rapidly: 0.7217
  swiftly: 0.7197
  eventually: 0.7043
 

# Problem 3 (30 points)


We have three tokens in the sentence **“bagel with cheese”**:

- $w_1 =$ *bagel*
- $w_2 =$ *with*
- $w_3 =$ *cheese*

with query, key, value vectors

$$
\begin{aligned}
q_1 &= [1,2,3], & k_1 &= [1,1,1], & v_1 &= [2,0,1],\\
q_2 &= [2,3,2], & k_2 &= [0,0,0], & v_2 &= [3,0,0],\\
q_3 &= [5,6,7], & k_3 &= [2,2,0], & v_3 &= [1,2,2].
\end{aligned}
$$

We want the self-attention output $z_1$ for token $w_1$ (*bagel*).

---

### (a) Unnormalized attention scores for $w_1$

First compute dot products $q_1 \cdot k_j$:

$$
\begin{aligned}
a_{11} &= q_1 \cdot k_1 = 1\cdot1 + 2\cdot1 + 3\cdot1 = 6,\\[2pt]
a_{12} &= q_1 \cdot k_2 = 1\cdot0 + 2\cdot0 + 3\cdot0 = 0,\\[2pt]
a_{13} &= q_1 \cdot k_3 = 1\cdot2 + 2\cdot2 + 3\cdot0 = 2 + 4 + 0 = 6.
\end{aligned}
$$

Scale by $\sqrt{\lvert k_1 \rvert}$.  
The key vectors have dimension $3$, so $\lvert k_1 \rvert = 3$, hence

$$
\sqrt{\lvert k_1 \rvert} = \sqrt{3} \approx 2 \quad (\text{given to round to } 2).
$$

Thus

$$
\begin{aligned}
a_{11} &= \frac{6}{2} = 3,\\
a_{12} &= \frac{0}{2} = 0,\\
a_{13} &= \frac{6}{2} = 3.
\end{aligned}
$$

---

### (b) Normalize attention weights for row $i=1$

Compute the row sum

$$
\sum_{k} a_{1k} = 3 + 0 + 3 = 6.
$$

Then

$$
\begin{aligned}
\alpha_{11} &= \frac{a_{11}}{\sum_k a_{1k}} = \frac{3}{6} = 0.5,\\[2pt]
\alpha_{12} &= \frac{a_{12}}{\sum_k a_{1k}} = \frac{0}{6} = 0,\\[2pt]
\alpha_{13} &= \frac{a_{13}}{\sum_k a_{1k}} = \frac{3}{6} = 0.5.
\end{aligned}
$$

So the normalized attention weights for $w_1$ are

$$
\alpha_{1\cdot} = [0.5,\; 0,\; 0.5].
$$

---

### (c) Weighted sum of value vectors

Now compute

$$
z_1 = \sum_{j=1}^3 \alpha_{1j} v_j
    = \alpha_{11} v_1 + \alpha_{12} v_2 + \alpha_{13} v_3.
$$

$$
\begin{aligned}
\alpha_{11} v_1 &= 0.5 [2,0,1] = [1,0,0.5],\\[2pt]
\alpha_{12} v_2 &= 0 [3,0,0] = [0,0,0],\\[2pt]
\alpha_{13} v_3 &= 0.5 [1,2,2] = [0.5,1,1].
\end{aligned}
$$

Add them:

$$
z_1 = [1,0,0.5] + [0,0,0] + [0.5,1,1] = [1.5,\; 1,\; 1.5].
$$

---

**Final answer**

$$
\boxed{z_1 = [1.5,\; 1,\; 1.5]}
$$

for the token *bagel*.
